In [32]:
import pandas as pd
from config import AppConfig
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [21]:
class StudentDataProcessor:
    """
    Classe responsável pelo carregamento e pré-processamento dos dados dos estudantes.
    """
    def __init__(self, filepath):
        self.filepath = filepath
        self.df = None
        self.X = None
        self.y = None

    def load_data(self):
        """Carrega o arquivo CSV para um DataFrame pandas."""
        try:
            self.df = pd.read_csv(self.filepath)
            print(f"Dados carregados! Dimensões: {self.df.shape}")
        except FileNotFoundError:
            print(f"Erro: Arquivo {self.filepath} não encontrado.")

    def preprocess_data(self, target_column='G3'):
        """
        Prepara os dados para treinamento:
        1. Remove colunas que vazam informação do futuro (G1, G2).
        2. Converte variáveis categóricas em numéricas (One-Hot Encoding).
        """
        if self.df is None:
            raise ValueError("O DataFrame está vazio. Execute load_data() primeiro.")

        # Separamos as Features (X) do Target (y)
        # Removemos G1 e G2 pois elas são notas parciais que facilitariam demais a previsão de G3
        X = self.df.drop([target_column, 'G1', 'G2'], axis=1)
        y = self.df[target_column]

        # Identifica colunas que são texto (object) para converter
        categorical_cols = X.select_dtypes(include=['object']).columns

        # Aplica One-Hot Encoding (transforma categorias em colunas de 0 e 1)
        X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

        self.X = X
        self.y = y
        print("Pré-processamento concluído.")
        return X, y

    def split_data(self, test_size=0.2, random_state=42):
        """Divide os dados em conjuntos de treino e teste."""
        X_train, X_test, y_train, y_test = train_test_split(
            self.X, self.y, test_size=test_size, random_state=random_state
        )
        print(f"Dados divididos. Treino: {X_train.shape}, Teste: {X_test.shape}")
        return X_train, X_test, y_train, y_test

In [23]:
class GradePredictor:
    """
    Classe responsável por criar, treinar e avaliar o modelo preditivo.
    """
    def __init__(self, model_type='linear'):
        self.model_type = model_type
        self.model = self._get_model()

    def _get_model(self):
        """Método interno (privado) para selecionar o algoritmo."""
        if self.model_type == 'linear':
            return LinearRegression()
        elif self.model_type == 'random_forest':
            return RandomForestRegressor(n_estimators=100, random_state=42)
        else:
            raise ValueError("Tipo de modelo não suportado. Use 'linear' ou 'random_forest'.")

    def train(self, X_train, y_train):
        """Treina o modelo com os dados fornecidos."""
        self.model.fit(X_train, y_train)
        print(f"Modelo ({self.model_type}) treinado com sucesso.")

    def predict(self, X_test):
        """Faz previsões com novos dados."""
        return self.model.predict(X_test)

    def evaluate(self, y_test, predictions):
        """Calcula e exibe métricas de erro."""
        mse = mean_squared_error(y_test, predictions)
        r2 = r2_score(y_test, predictions)
        
        print(f"\nAvaliação do Modelo ({self.model_type}):")
        print(f"   Erro Médio Quadrático (MSE): {mse:.2f}")
        print(f"   R2 Score (Precisão): {r2:.2f}")
        return mse, r2

In [37]:
# --- Bloco Principal de Execução (Main) ---
if __name__ == "__main__":
    # 1. Instanciar objeto de processamento
    csv_path = AppConfig.get_dataset_path()
    processor = StudentDataProcessor(csv_path)
    
    # 2. Executar pipeline de dados
    processor.load_data()
    processor.preprocess_data()
    X_train, X_test, y_train, y_test = processor.split_data()

    print("-" * 30)

    # 3. Testar com Regressão Linear (Conceito de Polimorfismo: mudamos o comportamento com um parâmetro)
    predictor_lr = GradePredictor(model_type='linear')
    predictor_lr.train(X_train, y_train)
    preds_lr = predictor_lr.predict(X_test)
    predictor_lr.evaluate(y_test, preds_lr)

    print("-" * 30)

    # 4. Testar com Random Forest (Um modelo mais robusto)
    predictor_rf = GradePredictor(model_type='random_forest')
    predictor_rf.train(X_train, y_train)
    preds_rf = predictor_rf.predict(X_test)
    predictor_rf.evaluate(y_test, preds_rf)

ValueError: Erro: Variável DATASET não encontrado no .env